In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 122.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/email_train_final.csv",split="train")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['input', 'output', 'x_train_len', 'instruction', 'text'],
    num_rows: 396
})

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [9]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/396 [00:00<?, ? examples/s]

In [10]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [11]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.235800
20,1.835500
30,1.229700
40,1.025300
50,0.663600
60,0.500400
70,0.395000
80,0.333200
90,0.210100
100,0.236800


TrainOutput(global_step=100, training_loss=0.9665307879447937, metrics={'train_runtime': 643.3696, 'train_samples_per_second': 2.487, 'train_steps_per_second': 0.155, 'total_flos': 2844956636774400.0, 'train_loss': 0.9665307879447937, 'epoch': 4.04})

In [12]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [13]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [29]:
text = '''[INST]<<SYS>>
 You are a helpful  assistant that provides information based on the given query in context.
<</SYS>>
Consider the domain knowledge based on the model is finetuned as its fine tuned on Gmail prediction and then carefully perform the task as Predict the next sentence based on given words or sentence.: scott thanks for the email have	 [/INST]'''

In [30]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [31]:
inputs = tokenizer(text, return_tensors="pt").to(device)

In [32]:
outputs = model.generate(**inputs, max_new_tokens=200)

In [33]:

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST]<<SYS>>
 You are a helpful  assistant that provides information based on the given query in context.
<</SYS>>
Consider the domain knowledge based on the model is finetuned as its fine tuned on Gmail prediction and then carefully perform the task as Predict the next sentence based on given words or sentence.: scott thanks for the email have	 [/INST]  Sure, I'd be happy to help! Based on the given words "Scott" and "thanks," it is likely that the next sentence will be a response from Scott to the email he received. Here are some possible sentences that could come next:

* "You're welcome, it was my pleasure to send you the email."
* "I appreciate your kind words, thank you for reaching out to me."
* "I'm glad you found the email helpful, let me know if you have any other questions."
* "Thank you for your email, I'll be sure to look into it."
* "I'm glad to hear from you, how can I assist you further?"

Please let me know if you would like me to generate more possibilities or if you

In [34]:
model.push_to_hub("llama2-chat-hub-my-finetuned-model-gmailComposer")

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sakil/llama2-chat-hub-my-finetuned-model-gmailComposer/commit/05512b2385d7d9d5cbded6a3429ad517b6c90b35', commit_message='Upload model', commit_description='', oid='05512b2385d7d9d5cbded6a3429ad517b6c90b35', pr_url=None, pr_revision=None, pr_num=None)

Inferencing:


In [40]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [35]:
lora_config = LoraConfig.from_pretrained('Sakil/llama2-chat-hub-my-finetuned-model-gmailComposer')
model = get_peft_model(model, lora_config)

adapter_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

In [45]:
def get_next_text(text):

    """This is working summarization prompt"""
    sys_instruction = "[INST]<<SYS>>You are a helpful assistant that provides information based on the given query in context<</SYS>>"
    input_text = sys_instruction + "Consider the domain knowledge based on the model is finetuned as its fine tuned on Gmail prediction and then carefully perform the task as Predict the next sentence based on given words or sentence.:" + text + "[/INST]"
#     print(input_text)

    device = "cuda:0"
    inputs = tokenizer(input_text, return_tensors="pt",return_token_type_ids=False).to(device)
    # inputs = tokenizer(input_text, return_tensors="pt").to(device)
    # inputs = tokenizer(input_text, return_tensors="pt").to(device)
#     outputs = model.generate(**inputs,min_length=50,max_new_tokens=200,early_stopping=True)
    outputs = model.generate(**inputs,max_new_tokens=200,temperature=0.8,top_p=0.95,top_k=50)
    summarized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Remove the system instruction from the summarized text
    return summarized_text
    # return summarized_text.split("[/INST]")[-1]

In [46]:
text_input='''
hey any good set available for
'''

In [47]:
final_result=get_next_text(text_input)

In [48]:
final_result

'[INST]<<SYS>>You are a helpful assistant that provides information based on the given query in context<</SYS>>Consider the domain knowledge based on the model is finetuned as its fine tuned on Gmail prediction and then carefully perform the task as Predict the next sentence based on given words or sentence.:\nhey any good set available for\n[/INST]  Sure, I\'d be happy to help! Given the context of the sentence "Hey any good set available for," I would predict the next sentence to be:\n\n"...purchasing or streaming."\n\nThis is based on the domain knowledge that "set" is a common term used in the context of music or entertainment, and the phrase "available for" typically implies a reference to a specific set of options or resources.'